In [43]:
import sys
import requests
import numpy as np
import pandas as pd
from io import StringIO
from pandas import json_normalize

from pathlib import Path

sys.path.append(str(Path.cwd().parent / 'scripts'))

from _helpers import to_date_period, to_datetime
from _elexon_helpers import robust_request
from build_base import build_physical_notifications_period

In [14]:
date, period = '2021-01-10', 24

In [32]:
accepts_url = (
    'https://data.elexon.co.uk/bmrs/api/v1/balancing/acceptances/' +
    'all?settlementDate={}&settlementPeriod={}&format=csv'
    )

# prices_url = (
#     "https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/"
#     "indicative/volumes/all/{}/{}/{}?{}&format=json"
# )

bidsoffers_url = (
    "https://data.elexon.co.uk/bmrs/api/v1/balancing/settlement/"
    "indicative/volumes/all/{}/{}/{}?format=json"
)

trades_url = (
        "https://data.elexon.co.uk/bmrs/api/v1/balancing/bid-offer/" +
        "all?settlementDate={}&settlementPeriod={}&format=csv".format(date, period)
    )

In [ ]:
response = robust_request(requests.get, accepts_url.format(date, period))
accepts = pd.read_csv(StringIO(response.text))
print(f"Retrieved {len(accepts)} balancing actions.")

In [22]:
accepts.head()

,SettlementDate,SettlementPeriodFrom,SettlementPeriodTo,TimeFrom,TimeTo,LevelFrom,LevelTo,NationalGridBmUnit,BmUnit,AcceptanceNumber,AcceptanceTime,DeemedBoFlag,SoFlag,StorFlag,RrFlag
0,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,0,90,FFES-4,T_FFES-4,153149,2021-01-10T11:58:00Z,False,False,False,False
1,2021-01-10,24,25,2021-01-10T11:59:00Z,2021-01-10T12:00:00Z,150,0,FOYE-2,T_FOYE-2,138919,2021-01-10T11:55:00Z,False,False,False,False
2,2021-01-10,24,24,2021-01-10T11:57:00Z,2021-01-10T11:59:00Z,150,150,FOYE-2,T_FOYE-2,138919,2021-01-10T11:55:00Z,False,False,False,False
3,2021-01-10,24,24,2021-01-10T11:57:00Z,2021-01-10T11:58:00Z,150,0,FOYE-2,T_FOYE-2,138918,2021-01-10T11:53:00Z,False,False,False,False
4,2021-01-10,24,25,2021-01-10T11:56:00Z,2021-01-10T12:01:00Z,650,666,DIDCB6,T_DIDCB6,151644,2021-01-10T11:21:00Z,False,False,False,False


In [23]:
unit_insert = "bmUnit={}"
accepted_units = accepts["NationalGridBmUnit"].unique().tolist()

insertion = "&".join(unit_insert.format(unit) for unit in accepted_units)

In [24]:
offers = json_normalize(
    robust_request(requests.get, bidsoffers_url.format('offer', date, period)).json()['data']
)

In [33]:
response = requests.get(trades_url)
trades = pd.read_csv(StringIO(response.text))

In [35]:
bm = 'WBURB-1'

In [36]:
offers.loc[offers.nationalGridBmUnit == bm].drop(columns=[col for col in offers.columns if 'pair' in col]).head()

,createdDateTime,settlementDate,settlementPeriod,startTime,bmUnit,bmUnitType,leadPartyName,nationalGridBmUnit,dataType,totalVolumeAccepted
64,2021-01-10T12:22:40Z,2021-01-10,24,2021-01-10T11:30:00Z,T_WBURB-1,T,West Burton B Limited,WBURB-1,Original,0.0
65,2021-01-10T12:22:40Z,2021-01-10,24,2021-01-10T11:30:00Z,T_WBURB-1,T,West Burton B Limited,WBURB-1,Original-Priced,0.0
66,2021-01-10T12:22:40Z,2021-01-10,24,2021-01-10T11:30:00Z,T_WBURB-1,T,West Burton B Limited,WBURB-1,Re-priced,0.0
67,2021-01-10T12:22:40Z,2021-01-10,24,2021-01-10T11:30:00Z,T_WBURB-1,T,West Burton B Limited,WBURB-1,Tagged,0.0


In [37]:
accepts.loc[accepts.NationalGridBmUnit == bm]

,SettlementDate,SettlementPeriodFrom,SettlementPeriodTo,TimeFrom,TimeTo,LevelFrom,LevelTo,NationalGridBmUnit,BmUnit,AcceptanceNumber,AcceptanceTime,DeemedBoFlag,SoFlag,StorFlag,RrFlag
6,2021-01-10,24,24,2021-01-10T11:56:00Z,2021-01-10T11:57:00Z,405,425,WBURB-1,T_WBURB-1,96444,2021-01-10T11:23:00Z,False,False,False,False
9,2021-01-10,24,25,2021-01-10T11:53:00Z,2021-01-10T12:23:00Z,405,405,WBURB-1,T_WBURB-1,96445,2021-01-10T11:51:00Z,False,False,False,False
45,2021-01-10,23,24,2021-01-10T11:26:00Z,2021-01-10T11:56:00Z,405,405,WBURB-1,T_WBURB-1,96444,2021-01-10T11:23:00Z,False,False,False,False


In [40]:
trades.loc[trades.NationalGridBmUnit == bm]

,SettlementDate,SettlementPeriod,NationalGridBmUnit,BmUnit,TimeFrom,TimeTo,LevelFrom,LevelTo,Bid,Offer,PairId
805,2021-01-10,24,WBURB-1,T_WBURB-1,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,-500,-500,0.5,72.5,-3
806,2021-01-10,24,WBURB-1,T_WBURB-1,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,-105,-105,46.2,72.5,-2
807,2021-01-10,24,WBURB-1,T_WBURB-1,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,-170,-170,46.2,72.5,-1
808,2021-01-10,24,WBURB-1,T_WBURB-1,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,18,18,46.2,72.5,1
809,2021-01-10,24,WBURB-1,T_WBURB-1,2021-01-10T11:30:00Z,2021-01-10T12:00:00Z,500,500,46.2,72.5,2


In [47]:
pn = build_physical_notifications_period(date, period)
pn.loc[[bm]]

NationalGridBmUnit
WBURB-1    425.0
Name: 2021-01-10 11:30:00+00:00, dtype: float64